![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 3 Notebook - AutoAI Notebook v1.15.2

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a> 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.7 and scikit-learn 0.23.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - xgboost.


In [2]:
!conda remove --force libxgboost -y
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.12.9 | tail -n 1
!pip install -U scikit-learn==0.23.2 | tail -n 1
!pip install -U xgboost==1.3.3 | tail -n 1


PackagesNotFoundError: The following packages are missing from the target environment:
  - libxgboost


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import in

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [3]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import S3Connection, S3Location

training_data_reference = [
    DataConnection(
    connection=S3Connection(
        api_key='nG1CZUkGQQcYUmMwKfOOMvlItbY3y0DFtF7hmPlS0WNw',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu.cloud-object-storage.appdomain.cloud'
    ),
        location=S3Location(
            bucket='cropyield-donotdelete-pr-rgrogyvcccq1z0',
            path='data.csv'
        )
    ),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='nG1CZUkGQQcYUmMwKfOOMvlItbY3y0DFtF7hmPlS0WNw',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu.cloud-object-storage.appdomain.cloud'
    ),
    location=S3Location(
        bucket='cropyield-donotdelete-pr-rgrogyvcccq1z0',
        path='auto_ml/171680e6-78d0-4bb7-bbf9-a5fb5bb6c08a/wml_data/bf836fcd-3104-43d5-b0d8-3b42f7e980de/data/automl',
        model_location='auto_ml/171680e6-78d0-4bb7-bbf9-a5fb5bb6c08a/wml_data/bf836fcd-3104-43d5-b0d8-3b42f7e980de/data/automl/cognito_output/Pipeline1/model.pickle',
        training_status='auto_ml/171680e6-78d0-4bb7-bbf9-a5fb5bb6c08a/wml_data/bf836fcd-3104-43d5-b0d8-3b42f7e980de/training-status.json'
    )
)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ibm_watson_machine_learning/helpers/connections/connections.py:792: UserWarning: S3 DataConnection is depreciated! Please use data_asset_id instead.
  warn(message="S3 DataConnection is depreciated! Please use data_asset_id instead.")


Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [4]:
experiment_metadata = dict(
    prediction_type='regression',
    prediction_column='PRODUCTION-(TONNES)',
    holdout_size=0.1,
    scoring='neg_root_mean_squared_error',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_reference=training_data_reference,
    training_result_reference=training_result_reference,
    deployment_url='https://eu-gb.ml.cloud.ibm.com',
    project_id='daa42e84-4674-40fe-b925-91e1f4917f08',
    drop_duplicates=True
)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.  
**Note**: If reading data using Flight Server for data connections results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [5]:
df = training_data_reference[0].read(experiment_metadata=experiment_metadata)
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ibm_watson_machine_learning/helpers/connections/connections.py:511: UserWarning: S3 DataConnection is depreciated! Please use data_asset_id instead.
  warn(message="S3 DataConnection is depreciated! Please use data_asset_id instead.")


<a id="split"></a>
##  Train and test data split

In [6]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['holdout_size'],
                                                    random_state=experiment_metadata['random_state'])

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [7]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA2
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TGen
import autoai_libs.cognito.transforms.transform_extras
from xgboost import XGBRegressor

#### Pre-processing & Estimator.

In [8]:
numpy_column_selector_0 = NumpyColumnSelector(columns=[0, 1, 2, 3, 4])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=["char_str", "char_str", "int_num", "int_num", "int_num"],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[5])
float_str2_float = FloatStr2Float(
    dtypes_list=["int_num"], missing_values_reference_list=[]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0, permutation_indices=[0, 1, 2, 3, 4, 5]
)
ta2 = TA2(
    fun=np.true_divide,
    name="divide",
    datatypes1=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints1=[autoai_libs.utils.fc_methods.is_not_categorical],
    datatypes2=[
        "intc", "intp", "int_", "uint8", "uint16", "uint32", "uint64", "int8",
        "int16", "int32", "int64", "short", "long", "longlong", "float16",
        "float32", "float64",
    ],
    feat_constraints2=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "DISTRICT", "CROP", "YEAR", "AVG-TEMP", "RAINFALL-(MM)",
        "AREA-(HECTARE)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 6),
    additional_col_count_to_keep=8,
    ptype="regression",
)
t_gen = TGen(
    fun=autoai_libs.cognito.transforms.transform_extras.NXOR,
    name="nxor",
    arg_count=2,
    datatypes_list=[["numeric"], ["numeric"]],
    feat_constraints_list=[
        [autoai_libs.utils.fc_methods.is_not_categorical],
        [autoai_libs.utils.fc_methods.is_not_categorical],
    ],
    col_names=[
        "DISTRICT", "CROP", "YEAR", "AVG-TEMP", "RAINFALL-(MM)",
        "AREA-(HECTARE)", "divide(DISTRICT__RAINFALL-(MM))",
        "divide(DISTRICT__AREA-(HECTARE))", "divide(RAINFALL-(MM)__DISTRICT)",
        "divide(RAINFALL-(MM)__AREA-(HECTARE))",
        "divide(AREA-(HECTARE)__DISTRICT)",
        "divide(AREA-(HECTARE)__RAINFALL-(MM))",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 6),
    additional_col_count_to_keep=8,
    ptype="regression",
)
xgb_regressor = XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    gpu_id=-1,
    interaction_constraints="",
    learning_rate=0.300000012,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=float("nan"),
    monotone_constraints="()",
    n_estimators=100,
    num_parallel_tree=1,
    random_state=33,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    subsample=1,
    tree_method="exact",
    validate_parameters=1,
    verbosity=0,
    silent=False,
    nthread=1,
    seed=33,
)


#### Pipeline.

In [9]:
pipeline = make_pipeline(
    union, numpy_permute_array, ta2, fs1_0, t_gen, fs1_1, xgb_regressor
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [10]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [11]:
pipeline.fit(train_X,train_y)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/sklearn/base.py:693: RuntimeWarning: divide by zero encountered in true_divide
  return self.fit(X, y, **fit_params).transform(X)
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/sklearn/base.py:693: RuntimeWarning: invalid value encountered in true_divide
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('pipeline-1',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               4])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                          

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [12]:
score = scorer(pipeline, test_X, test_y)
print(score)

-1563.3471984573375


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/sklearn/pipeline.py:407: RuntimeWarning: divide by zero encountered in true_divide
  Xt = transform.transform(Xt)


### Calling the `predict` method
If you want to get a prediction using pipeline model object, call `pipeline.predict()`.

**Note:** If you want to work with pure sklearn model:
 - add the following parameter to `get_pipeline` call: `astype='sklearn'`,
 - or `scikit_learn_pipeline = pipeline.export_to_sklearn_pipeline()`

In [13]:
pipeline.predict(test_X)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/sklearn/pipeline.py:407: RuntimeWarning: divide by zero encountered in true_divide
  Xt = transform.transform(Xt)


array([30963.262 , 29964.674 , 26202.877 , 15987.352 , 27984.254 ,
       25676.494 , 15525.855 , 28943.08  , 24740.195 , 16371.305 ,
       29972.316 , 25098.756 , 16529.25  , 29330.365 , 35630.582 ,
       16315.354 , 28706.385 , 33285.777 , 25383.102 , 17144.203 ,
       29319.465 , 31184.285 , 30163.877 , 35953.8   , 26642.572 ,
       28454.564 , 17465.193 , 16815.754 , 29855.73  , 26020.36  ,
       15485.663 , 33670.32  , 17563.572 , 28294.969 , 16819.523 ,
       28708.648 , 31218.96  , 24699.451 , 15712.31  , 30274.46  ,
       24349.398 , 31309.459 , 32564.201 , 31728.508 , 35332.9   ,
       28350.504 , 29985.668 , 27800.68  , 26646.045 , 27100.924 ,
       36979.594 , 16980.154 , 16362.8545, 28584.586 , 28587.428 ,
       27124.906 , 27204.336 , 15706.45  , 31641.447 , 15539.366 ,
       31728.193 , 29845.146 , 35443.35  , 15702.282 , 17061.975 ,
       29284.994 , 16138.209 , 17081.498 , 28702.783 , 15435.393 ,
       28514.977 , 28061.574 , 31104.055 , 30645.29  , 24934.7

<a id="next_steps"></a>
# Next steps

#### [Model deployment as webservice](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/deployments/autoai)
#### [Run AutoAI experiment with python SDK](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)  

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2021 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___